In [7]:
from threading import RLock
from time import sleep
from threading import Thread
from sys import stdout

class Queue:
    
    def __init__(self):
        self.__items = []
        self.__lock = RLock()

    def enqueue(self, item):
        self.__lock.acquire()
        self.__items.append(item)
        self.__lock.release()
        
    def is_empty(self):
        self.__lock.acquire()
        empty = len(self.__items) == 0
        self.__lock.release()
        return empty
    
    def dequeue(self):
        self.__lock.acquire()
        if self.is_empty():
            raise RuntimeError
        item = self.__items[0]
        self.__items = self.__items[1:]
        self.__lock.release()
        return item
    
    def lock(self):
        self.__lock.acquire()
    
    def unlock(self):
        self.__lock.release()
    
def producer(queue):
    for i in range(10):
        print('producer puts', i, 'in queue')
        stdout.flush()
        queue.enqueue(i)
        sleep(0.4)

def consumer(queue, name):
    for i in range(10):
        queue.lock()
        if not queue.is_empty():
            print('consumer', name, 'gets', queue.dequeue())
            stdout.flush()
        queue.unlock()
        sleep(0.5)

In [8]:
queue = Queue()
prod = Thread(target=producer, args=(queue,))
cons1 = Thread(target=consumer, args=(queue,'1'))
cons2 = Thread(target=consumer, args=(queue,'2'))

prod.start()
cons1.start()
cons2.start()
prod.join()
cons1.join()
cons2.join()

producer puts 0 in queue
producer puts 1 in queue
consumer 1 gets 0
consumer 2 gets 1
producer puts 2 in queue
consumer 1 gets 2
producer puts 3 in queue
consumer 1 gets 3
producer puts 4 in queue
consumer 1 gets 4
producer puts 5 in queue
producer puts 6 in queue
consumer 1 gets 5
consumer 2 gets 6
producer puts 7 in queue
consumer 1 gets 7
producer puts 8 in queue
consumer 1 gets 8
producer puts 9 in queue
consumer 1 gets 9
